In [ ]:
import requests
import numpy as np
import os, sys
import math
import arrow
import pytz
from datetime import datetime, date, timedelta
from timezonefinder import TimezoneFinder
from matplotlib import pyplot as plt, dates
from matplotlib.ticker import *
from matplotlib_helper import *

In [ ]:
M_PUBLIC_CLOUD_LOCATION = {
    ('AWS', 'us-west-1'): (37.00578, -121.56828),
    ('AWS', 'us-west-2'): (45.840410, -119.289460),
    ('AWS', 'us-east-1'): (39.983334, -82.983330),
    ('AWS', 'us-east-2'): (39.040283, -77.485165),
}
def get_location_for_public_cloud(cloud_vendor, region):
    '''Looks up the GPS coordinate for public cloud region.'''
    if (cloud_vendor, region) in M_PUBLIC_CLOUD_LOCATION:
        return M_PUBLIC_CLOUD_LOCATION[(cloud_vendor, region)]
    else:
        return (math.nan, math.nan)

In [ ]:
def plot_timeseries(data_array, plot_axis=None, timestamp_column_name='timestamp', prefix=None, use_relative_time=False, color=None, index=0):
    x = [entry[timestamp_column_name] for entry in data_array]
    if use_relative_time:
        start_time = x[0]
        x = [(t - start_time).total_seconds() for t in x]
    data_keys = []
    for key in data_array[0].keys():
        if key == timestamp_column_name:
            continue
        data_keys.append(key)
    lines = []
    for key in data_keys:
        data_series = [entry[key] for entry in data_array]
        label = (('%s - ' % prefix if prefix else '') + key) if len(data_keys) > 1 else (prefix if prefix else '')
        if plot_axis is None:
            plot_axis = plt.gca()
        line = plot_axis.plot(x, data_series, color=color, linestyle=get_linestyle(index), label=label)
        index += 1
        lines.append(line)
    return lines

In [ ]:
def plot_carbon_intensity_range(cloud_vendor, region, date:date = None, timerange:timedelta = timedelta(weeks=1), use_utc_time_of_day = True):
    print(cloud_vendor, region)
    url_get_carbon_intensity = 'http://yeti-09.sysnet.ucsd.edu/carbon-intensity/'
    (latitude, longitude) = get_location_for_public_cloud(cloud_vendor, region)
    if date is None:
        date = arrow.get().shift(weeks=-1).date()
    if use_utc_time_of_day:
        timezone = pytz.UTC
    else:
        timezone_str = TimezoneFinder().timezone_at(lng=longitude, lat=latitude)
        timezone = pytz.timezone(timezone_str)
    date = arrow.get(date, tzinfo=timezone)
    # print(timezone_str, date, file=sys.stderr)
    response = requests.get(url_get_carbon_intensity, params={
        'latitude': latitude,
        'longitude': longitude,
        'start': date,
        'end': date.shift(minutes=-1) + timerange,
    })
    assert response.ok, "Carbon intensity lookup failed (%d): %s" % (response.status_code, response.text)
    response_json = response.json()
    electricity_region = response_json['region']
    print('region:', electricity_region)
    carbon_intensities = response_json['carbon_intensities']
    data_for_plot = []
    l_carbon_intensity = []
    print(carbon_intensities[0], carbon_intensities[-1], file=sys.stderr)
    for element in carbon_intensities:
        timestamp = arrow.get(element['timestamp']).datetime
        carbon_intensity = float(element['carbon_intensity'])
        data_for_plot.append({
            'timestamp': timestamp,
            'carbon_intensity': carbon_intensity,
        })
        l_carbon_intensity.append(carbon_intensity)
    plot_timeseries(data_for_plot, use_relative_time=False, prefix=f'{cloud_vendor} {region} ({electricity_region})')
    print('Avg/Min/Max carbon intensity: %.2f/%.2f/%.2f' % (
        np.mean(l_carbon_intensity),
        np.min(l_carbon_intensity),
        np.max(l_carbon_intensity),
    ))

In [ ]:
use_utc_time_of_day = True

# plt.figure(figsize=(10, 6))
all_cloud_vendor_and_regions = [
    ('AWS', 'us-west-1'),
    ('AWS', 'us-west-2'),
    ('AWS', 'us-east-1'),
    # AWS us-east-2 uses the same ISO as us-east-1
    # ('AWS', 'us-east-2'),
]

stepsize = timedelta(weeks=1)
for offset in range(4):
    target_date = arrow.get(datetime(2022, 6, 1)) + (stepsize * -offset)
    print(f'\n{stepsize.days}-day starting {target_date.strftime("%Y-%m-%d")}')
    plt.figure(figsize=(8,4))
    for (cloud_vendor, region) in all_cloud_vendor_and_regions:
        plot_carbon_intensity_range(cloud_vendor, region, date=target_date, timerange=stepsize, use_utc_time_of_day=use_utc_time_of_day)
    if stepsize.total_seconds() == timedelta(days=1).total_seconds():
        date_formatter_string = "%H:%M"
        xlabel = f'Time of day ({"UTC" if use_utc_time_of_day else "local"})'
    else:
        date_formatter_string = "%Y/%m/%d %H:%M"
        xlabel = 'Date'
    ax = plt.gca()
    ax.xaxis.set_major_formatter(dates.DateFormatter(date_formatter_string))
    plt.xlabel(xlabel)
    plt.ylabel('Carbon intensity (gCO2/kWh)')
    plt.title(f'{stepsize.days}-day carbon intensity of {cloud_vendor} {region} starting {target_date.strftime("%Y-%m-%d")}')
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.xticks(rotation=30)
    plt.ylim(0, 800)
    plt.tight_layout()
    savefig_filename = 'carbon-intensity.%s.%ddays.png' % (target_date.strftime("%Y-%m-%d"), stepsize.days)
    plt.savefig(savefig_filename)